In [ ]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3

! pip install nbformat==5.9.2 

In [1]:
# load pandas to deal with the data
import pandas as pd
# plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# load data from table file where entries are separated with a space
data = pd.read_csv("../data/flickr_data2.csv", sep=",")
# show first 5 rows of the dataframe
data.head()

C:\Users\tomek\AppData\Local\Temp\ipykernel_16560\2003460992.py:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/flickr_data2.csv", sep=",")


,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,2010,23,20,28.0,2,2010.0,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17,28,2,2010,52,17,28.0,2,2010.0,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,2010,33,17,28.0,2,2010.0,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN


## Introduction

Analyse 1: Les attributs important semblent être lat, long et l'heure pour regrouper les personnes.

On remarque que les 3 derniéres colonnes sont remplies de NaN, nous allons explorer les détails de ces colonnes 
Les types de "date" et "minute" sont float et non int.

In [3]:
print(len(data))

420240


Il y a 420240 lignes.

## Nettoyage des données 

On évalue et nettoie les doublons 

In [4]:
print(len(data['id'].unique()))

168097


In [17]:
ligne_id_egales = len(data) - len(data['id'].unique())
ligne_completement_egales = len(data) - data.drop_duplicates().shape[0]

print(ligne_id_egales, ligne_completement_egales)


252143 232642


Il y a des id et des lignes dupliquées. 

On ne veut pas supprimer tous les doublons d'id car un doublon pourrait avoir plus d'information qu'un autre (ligne avec des attributs vides)

Comme attendu, ligne_completement_egales< ligne_id_egales.
Nous allons donc dans un premier temps supprimer les lignes entiérement dupliquées.

In [ ]:
data_cleaned_1 = data.drop_duplicates() #DataFrame sans doublons sur les lignes

On cherche à savoir si les lignes avec des id identiques sont différentes ou seulement trouées.


In [ ]:
duplicated_ids = data_cleaned_1.loc[data_cleaned_1['id'].duplicated(), 'id'].unique().tolist()

data_cleaned_1[data_cleaned_1['id'].isin(duplicated_ids)].to_csv('../temp/duplicated_ids.csv')

On voit que la plupart des doublons sont liés au données type str: Tags, Description. Les données importantes semblent identiques.

Nous allons supprimer les colonnes parasites + les 3 dernières colonnes.

In [ ]:
cols = ["tags", "title", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18"]
cols_to_drop = [c for c in cols if c in data_cleaned_1.columns]
print(cols, cols_to_drop)

['tags', 'title', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'] ['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']


On remarque qu'il y avait des espaces dans les colonnes tags et title. Nous allons corriger ça et afficher les mêmes données sans cols.

In [ ]:
cols = data_cleaned_1.columns.tolist()
data_cleaned_2 = data_cleaned_1.rename(columns={cols[4]:'tags', cols[5]: 'title'})

,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,2010,23,20,28.0,2,2010.0,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17,28,2,2010,52,17,28.0,2,2010.0,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,2010,33,17,28.0,2,2010.0,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,2010,38,12,28.0,2,2010.0,NaN,NaN,NaN


Nous pouvons continuer nos tests.

In [59]:
cols_to_drop = ["tags", "title", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18"]
data_parasites_dropped = data_cleaned_2.drop(columns=cols_to_drop)

data_parasites_dropped[data_parasites_dropped['id'].isin(duplicated_ids)].to_csv('../temp/data_parasites_dropped.csv')